## 

# 🎬 Movies Data Pipeline
Ce notebook contient un pipeline de traitement des données du fichier `TMDB_all_movies.csv`, dans le cadre d'un projet de data engineering.

## 1. Ingestion

In [210]:
from pyspark.sql import SparkSession
import pyspark
from pyspark.sql.functions import count, col, when, array, split, size, sum as _sum, row_number, mean, year
from pyspark.sql.window import Window
from pyspark.ml.feature import CountVectorizer, StandardScaler, VectorAssembler, Tokenizer, StopWordsRemover, HashingTF, IDF
from pyspark.ml import Pipeline

In [211]:
spark = SparkSession.builder.appName("Movie recommender").getOrCreate()

df = spark.read.csv("../data/TMDB_all_movies.csv", header=True, inferSchema=True, sep=",", quote='"', escape='"', multiLine=True)

In [212]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- vote_average: double (nullable = true)
 |-- vote_count: double (nullable = true)
 |-- status: string (nullable = true)
 |-- release_date: date (nullable = true)
 |-- revenue: double (nullable = true)
 |-- runtime: double (nullable = true)
 |-- budget: double (nullable = true)
 |-- imdb_id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: double (nullable = true)
 |-- tagline: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- spoken_languages: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- director: string (nullable = true)
 |-- director_of_photography: string (nullable = true)
 |-- writers: string (nullable = true)
 |-- producers: string (nu

In [213]:
# Type de chaque colonne :
    # vote_average: float
    # vote_count: int
    # release_date: date
    # revenue: float
    # runtime: float
    # budget: float
    # popularity: float
    # genres, production_countries, production_companies, spoken, cast, director, writers: string / one-hot encoded ?

cols_to_check = ["title", "original_title", "overview", "release_date", "genres", "production_countries", "production_companies", "spoken_languages", "cast", "director", "writers"]
df = df.withColumn(
    "completeness_score",
    sum([when(col(c).isNotNull(), 1).otherwise(0) for c in cols_to_check])
)

df = df.withColumn("vote_average", df["vote_average"].try_cast("double").try_cast("float")) \
    .withColumn("vote_count", df["vote_count"].try_cast("double").try_cast("int")) \
    .withColumn("release_date", df["release_date"].try_cast("date")) \
    .withColumn("revenue", df["revenue"].try_cast("double").try_cast("float")) \
    .withColumn("runtime", df["runtime"].try_cast("double").try_cast("float")) \
    .withColumn("budget", df["budget"].try_cast("double").try_cast("float")) \
    .withColumn("popularity", df["popularity"].try_cast("double").try_cast("float")) \
    .withColumn("genres_array", when(col("genres").isNotNull(), split(col("genres"), ",\\s*")).otherwise(array())) \
    .withColumn("production_countries_array", when(col("production_countries").isNotNull(), split(col("production_countries"), ",\\s*")).otherwise(array())) \
    .withColumn("production_companies_array", when(col("production_companies").isNotNull(), split(col("production_companies"), ",\\s*")).otherwise(array())) \
    .withColumn("spoken_languages_array", when(col("spoken_languages").isNotNull(), split(col("spoken_languages"), ",\\s*")).otherwise(array())) \
    .withColumn("cast_array", when(col("cast").isNotNull(), split(col("cast"), ",\\s*")).otherwise(array())) \
    .withColumn("director_array", when(col("director").isNotNull(), split(col("director"), ",\\s*")).otherwise(array())) \
    .withColumn("writers_array", when(col("writers").isNotNull(), split(col("writers"), ",\\s*")).otherwise(array())) \
    .withColumn("release_year", year("release_date").try_cast("double").try_cast("int")) \
    .withColumn("original_language_array", when(col("original_language").isNotNull(), split(col("original_language"), ",\\s*")).otherwise(array()))

df = df.drop("genres", "production_countries", "production_companies", "spoken_languages", "cast", "director", "writers", "original_language")

df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- vote_average: float (nullable = true)
 |-- vote_count: integer (nullable = true)
 |-- status: string (nullable = true)
 |-- release_date: date (nullable = true)
 |-- revenue: float (nullable = true)
 |-- runtime: float (nullable = true)
 |-- budget: float (nullable = true)
 |-- imdb_id: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: float (nullable = true)
 |-- tagline: string (nullable = true)
 |-- director_of_photography: string (nullable = true)
 |-- producers: string (nullable = true)
 |-- music_composer: string (nullable = true)
 |-- imdb_rating: double (nullable = true)
 |-- imdb_votes: double (nullable = true)
 |-- poster_path: string (nullable = true)
 |-- completeness_score: integer (nullable = false)
 |-- genres_array: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- production_countries

## 2. Exploration

In [214]:
df.show(5, truncate=False)

+---+--------------------------------+------------+----------+--------+------------+-----------+-------+---------+---------+--------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [215]:
# null values by columns
df.select([pyspark.sql.functions.count(pyspark.sql.functions.when(pyspark.sql.functions.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---+-----+------------+----------+------+------------+-------+-------+------+-------+--------------+--------+----------+-------+-----------------------+---------+--------------+-----------+----------+-----------+------------------+------------+--------------------------+--------------------------+----------------------+----------+--------------+-------------+------------+-----------------------+
| id|title|vote_average|vote_count|status|release_date|revenue|runtime|budget|imdb_id|original_title|overview|popularity|tagline|director_of_photography|producers|music_composer|imdb_rating|imdb_votes|poster_path|completeness_score|genres_array|production_countries_array|production_companies_array|spoken_languages_array|cast_array|director_array|writers_array|release_year|original_language_array|
+---+-----+------------+----------+------+------------+-------+-------+------+-------+--------------+--------+----------+-------+-----------------------+---------+--------------+-----------+----------

In [216]:
# overview null but vote_count > 5

# df.filter(
#     (df["overview"].isNull()) & (df["vote_count"] > 5)
# ).count()

df.filter(
    (df["overview"].isNull()) & (df["vote_count"] > 5)
).show(100, truncate=False)

+-----+---------------------------------------------------------------------------------------+------------+----------+--------+------------+--------+-------+---------+---------+-----------------------------------------------------------------------+--------+----------+---------------------------+----------------------------------------------------------------------------------------------+-----------------------------------------------------------+---------------------------------------------------------------------+-----------+----------+--------------------------------+------------------+------------------------------------------------+--------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------

In [217]:
# genres null but vote_count > 10
# df.filter(
#     (size(col("genres_array")) == 0) & (df["vote_count"] > 10)
# ).count()

# genres null
df.filter(
    size(col("genres_array")) == 0
).show(100, truncate=False)

+-----+-----------------------------------------------------------------------------+------------+----------+--------+------------+---------+-------+---------+---------+-----------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [218]:
# get duplicate titles
duplicate_titles = (
    df.groupBy("title") \
      .agg(count("*").alias("count"))\
      .filter(col("count") > 1)\
)

duplicate_titles.show(100)

+--------------------+-----+
|               title|count|
+--------------------+-----+
|       Heading South|    2|
|                Nell|    3|
|             Nemesis|   20|
|          Der Tunnel|    6|
|              Deszcz|    2|
|         Deep Rising|    2|
|    Straight to Hell|    2|
|Dance with the Devil|    4|
|              Room 6|    2|
|              Heaven|   30|
|                Silk|   14|
|       Crossing Over|   10|
|   What No One Knows|    2|
|        The Big Bang|    6|
|          Riverworld|    2|
|       Sugar & Spice|    4|
|            Deep Red|    4|
|        Miracle Mile|    2|
|             Larceny|    4|
|     My Name Is Khan|    2|
|        Generation X|    2|
|      A Woman's Face|    3|
|         It's a Gift|    2|
|    La Vie de Bohème|    2|
|             Amateur|   14|
|              Crisis|   17|
|  A Woman Is a Woman|    2|
|       Natural Enemy|    2|
|           Surprise!|   12|
|Diary of a Chambe...|    3|
|         Lesser Evil|    2|
|             

In [219]:
# get duplicate titles with all columns
duplicate_titles_full = (
    df.join(duplicate_titles, "title") \
      .select(df["title"], df["release_year"], df["completeness_score"], df["original_language_array"], df["original_title"], df["overview"], df["genres_array"], df["production_countries_array"], df["production_companies_array"], df["spoken_languages_array"], df["cast_array"], df["writers_array"], df["director_array"], duplicate_titles["count"])
)
duplicate_titles_full.show(100, truncate=False)

+----------------------------------------------------+------------+------------------+-----------------------+----------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [220]:
df.select("completeness_score").describe().show()

+-------+------------------+
|summary|completeness_score|
+-------+------------------+
|  count|           1118702|
|   mean| 8.100197371596726|
| stddev|2.5031281958716334|
|    min|                 0|
|    max|                11|
+-------+------------------+



In [221]:
# rows with completeness_score < 4
df.filter(
    df["completeness_score"] < 4
).show(100, truncate=False)

+-----+-------------------------------------------------------------+------------+----------+--------+------------+-------+-------+------+---------+-------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+--------------------+-----------------------+---------+--------------+-----------+----------+-------------------

In [222]:
# count rows who have duplicate titles and same release_date, with not null release_date
duplicate_titles_count = (
    df.groupBy("title", "release_date") \
        .agg(count("*").alias("count"))\
        .filter((col("count") > 1) & (col("release_date").isNotNull()))\
)
duplicate_titles_count.show(100, truncate=False)

+----------------------------------------------------+------------+-----+
|title                                               |release_date|count|
+----------------------------------------------------+------------+-----+
|Top Mission                                         |1987-01-01  |2    |
|Alpha                                               |1972-01-01  |2    |
|The Mystery Mausoleum                               |2018-10-27  |2    |
|Parental Guidance                                   |2002-06-12  |2    |
|Devi                                                |2024-04-28  |2    |
|Escape                                              |2024-07-03  |2    |
|Supper                                              |2022-06-02  |2    |
|Voorhees: Night of the Beast                        |2021-12-31  |2    |
|Fur Fatale                                          |2007-01-01  |2    |
|BROWN SUGAR SALMON                                  |2021-06-23  |2    |
|The Return                           

In [223]:
# release_date null
df.filter(
    df["release_date"].isNull()
).show(100, truncate=False)

+-----+----------------------------------------------------------------------+------------+----------+---------------+------------+-------+-------+---------+---------+----------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [224]:
# count rows with release_date, production_companies, production_countries, spoken_languages, cast, director and writers null
df.filter(
    df.release_date.isNull() &
    df.overview.isNull() &
    (size(col("production_companies_array")) == 0) &
    (size(col("production_countries_array")) == 0) &
    (size(col("spoken_languages_array")) == 0) &
    (size(col("cast_array")) == 0) &
    (size(col("director_array")) == 0) &
    (size(col("writers_array")) == 0) &
    (size(col("genres_array")) == 0)
).count()

21531

In [225]:
df.select("popularity", "vote_count", "vote_average", "revenue", "runtime", "budget").describe().show()

+-------+------------------+------------------+------------------+-------------------+-----------------+-----------------+
|summary|        popularity|        vote_count|      vote_average|            revenue|          runtime|           budget|
+-------+------------------+------------------+------------------+-------------------+-----------------+-----------------+
|  count|           1118700|           1118700|           1118700|            1118700|          1118700|          1118700|
|   mean|1.0115808506442796|22.373812460892108|2.0368384768215444|  776736.8174524001|44.47222311611692|303927.2866764995|
| stddev|2.5777876881508663| 358.9394960574097|3.0729850067899678|1.956196610053568E7|57.65227780411342|5361099.808076112|
|    min|               0.0|                 0|               0.0|              -12.0|              0.0|              0.0|
|    max|          658.8665|             37627|              10.0|              5.0E9|          14400.0|            1.0E9|
+-------+-------

In [226]:
# count values = 0 in popularity, vote_count, vote_average, revenue, runtime, budget
df.select(
    _sum(when(col("popularity") == 0, 1).otherwise(0)).alias("popularity_0"),
    _sum(when(col("vote_count") == 0, 1).otherwise(0)).alias("vote_count_0"),
    _sum(when(col("vote_average") == 0, 1).otherwise(0)).alias("vote_average_0"),
    _sum(when(col("revenue") == 0, 1).otherwise(0)).alias("revenue_0"),
    _sum(when(col("runtime") == 0, 1).otherwise(0)).alias("runtime_0"),
    _sum(when(col("budget") == 0, 1).otherwise(0)).alias("budget_0")
).show()

+------------+------------+--------------+---------+---------+--------+
|popularity_0|vote_count_0|vote_average_0|revenue_0|runtime_0|budget_0|
+------------+------------+--------------+---------+---------+--------+
|        7683|      739532|        739854|  1092512|   269552| 1047893|
+------------+------------+--------------+---------+---------+--------+



In [227]:
df.select("original_language_array").distinct().count()

179

## 3. Nettoyage

In [228]:
# enlever toutes les lignes qui ne sont pas en released
df = df.filter(df["status"] == "Released")
df.select("status").distinct().show()

+--------+
|  status|
+--------+
|Released|
+--------+



In [229]:
# drop columns that are not useful for the calculation
df = df.drop("status", "imdb_id", "tagline", "director_of_photography", "producers", "imdb_rating", "imdb_votes", "music_composer")
df.show(5, truncate=False)

+---+--------------------------------+------------+----------+------------+-----------+-------+---------+--------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [230]:
# drop rows that have no title
df = df.filter(df["title"].isNotNull() & (df["title"] != ""))

In [231]:
# replace null values in overview with empty string and release_year with median year
median_year = df.approxQuantile("release_year", [0.5], 0.01)[0]
df = df.fillna({
    'overview': '',
    'release_year': median_year
})

In [232]:
# drop rows who have completeness_score < 4
df = df.filter(df["completeness_score"] >= 4)

In [233]:
# drop rows with release_date, production_companies, production_countries, spoken_languages, cast, director, writers, overview and genres null
df = df.filter(
    ~(
        df.release_date.isNull() &
        df.overview.isNull() &
        (size(col("production_companies_array")) == 0) &
        (size(col("production_countries_array")) == 0) &
        (size(col("spoken_languages_array")) == 0) &
        (size(col("cast_array")) == 0) &
        (size(col("director_array")) == 0) &
        (size(col("writers_array")) == 0) &
        (size(col("genres_array")) == 0)
    )
)

In [234]:
# vérifier les doublons de titres avec release_date, et supprimer la ligne avec le completeness_score le plus bas
window = Window.partitionBy("title", "release_date").orderBy(col("completeness_score").desc())

# Garder la ligne la plus complète
df = df.withColumn("row_num", row_number().over(window)) \
       .filter(col("row_num") == 1) \
       .drop("row_num")

In [235]:
# vérifier les doublons de titres avec overview, et supprimer la ligne avec le completeness_score le plus bas
window = Window.partitionBy("title", "overview").orderBy(col("completeness_score").desc())

# Garder la ligne la plus complète
df = df.withColumn("row_num", row_number().over(window)) \
       .filter(col("row_num") == 1) \
       .drop("row_num", "completeness_score")

In [236]:
# set la moyenne de runtime si runtime = 0, same for release_year
mean_runtime = df.select(mean("runtime")).first()[0]
df = df.withColumn("runtime", when(col("runtime") == 0, mean_runtime).otherwise(col("runtime")))

# mean_release_year = df.select(mean("runtime")).first()[0]
# df = df.withColumn("runtime", when(col("runtime") == 0, mean_runtime).otherwise(col("runtime")))

In [237]:
# null values by columns 2
df.select([pyspark.sql.functions.count(pyspark.sql.functions.when(pyspark.sql.functions.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---+-----+------------+----------+------------+-------+-------+------+--------------+--------+----------+-----------+------------+--------------------------+--------------------------+----------------------+----------+--------------+-------------+------------+-----------------------+
| id|title|vote_average|vote_count|release_date|revenue|runtime|budget|original_title|overview|popularity|poster_path|genres_array|production_countries_array|production_companies_array|spoken_languages_array|cast_array|director_array|writers_array|release_year|original_language_array|
+---+-----+------------+----------+------------+-------+-------+------+--------------+--------+----------+-----------+------------+--------------------------+--------------------------+----------------------+----------+--------------+-------------+------------+-----------------------+
|  0|    0|           0|         0|       57635|      0|      0|     0|             0|       0|         0|     247964|           0|           

In [238]:
# split df to put id, title, oiginal_title, poster_path in a separate df
df_id_title = df.select("id", "title", "original_title", "poster_path", "genres_array", "production_countries_array", "production_companies_array", "spoken_languages_array", "cast_array", "director_array", "writers_array", "vote_count", "vote_average", "budget", "release_date", "runtime", "original_language_array", "popularity", "overview")
df = df.drop("title", "original_title", "poster_path", "vote_count", "vote_average", "budget", "revenue", "release_date")

In [239]:
df.show(5, truncate=True)

+-------+-----------------+--------------------+----------+-------------+--------------------------+--------------------------+----------------------+--------------------+------------------+--------------------+------------+-----------------------+
|     id|          runtime|            overview|popularity| genres_array|production_countries_array|production_companies_array|spoken_languages_array|          cast_array|    director_array|       writers_array|release_year|original_language_array|
+-------+-----------------+--------------------+----------+-------------+--------------------------+--------------------------+----------------------+--------------------+------------------+--------------------+------------+-----------------------+
|1301803|              4.0|An original short...|      0.09|[Documentary]|                 [Ukraine]|                        []|         [Український]|                  []|[Orest Smilianets]|                  []|        2023|                   [uk]|
|124

## 4. Transformation

In [240]:
# use CountVectorizer to vectorize the genres, production_countries, production_companies, spoken_languages, cast, director and writers columns
vectorizer_genres = CountVectorizer(inputCol="genres_array", outputCol="genres_vector")
vectorizer_production_countries = CountVectorizer(inputCol="production_countries_array", outputCol="production_countries_vector")
vectorizer_production_companies = CountVectorizer(inputCol="production_companies_array", outputCol="production_companies_vector")
vectorizer_spoken_languages = CountVectorizer(inputCol="spoken_languages_array", outputCol="spoken_languages_vector")
vectorizer_cast = CountVectorizer(inputCol="cast_array", outputCol="cast_vector")
vectorizer_director = CountVectorizer(inputCol="director_array", outputCol="director_vector")
vectorizer_writers = CountVectorizer(inputCol="writers_array", outputCol="writers_vector")
vectorizer_original_language = CountVectorizer(inputCol="original_language_array", outputCol="original_language_vector")

In [241]:
# 1. Tokenisation
tokenizer = Tokenizer(inputCol="overview", outputCol="overview_tokens")

# 2. Suppression des mots vides
remover = StopWordsRemover(inputCol="overview_tokens", outputCol="overview_filtered")

# 3. TF
hashingTF = HashingTF(inputCol="overview_filtered", outputCol="overview_tf", numFeatures=10000)

# 4. IDF
idf = IDF(inputCol="overview_tf", outputCol="overview_vector")

In [242]:
runtime_assembler = VectorAssembler(inputCols=["runtime"], outputCol="runtime_vec")
runtime_scaler = StandardScaler(inputCol="runtime_vec", outputCol="runtime_scaled")
popularity_assembler = VectorAssembler(inputCols=["popularity"], outputCol="popularity_vec")
popularity_scaler = StandardScaler(inputCol="popularity_vec", outputCol="popularity_scaled")
release_year_assembler = VectorAssembler(inputCols=["release_year"], outputCol="release_year_vec", handleInvalid="keep")
release_year_scaler = StandardScaler(inputCol="release_year_vec", outputCol="release_year_scaled")

In [243]:
pipeline = Pipeline(stages=[
    tokenizer,
    remover,
    hashingTF,
    idf,
    vectorizer_genres,
    vectorizer_production_countries,
    vectorizer_production_companies,
    vectorizer_spoken_languages,
    vectorizer_cast,
    vectorizer_director,
    vectorizer_writers,
    vectorizer_original_language,
    runtime_assembler,
    runtime_scaler,
    popularity_assembler,
    popularity_scaler,
    release_year_assembler,
    release_year_scaler
])
pipeline_model = pipeline.fit(df)
df_vec = pipeline_model.transform(df)

In [ ]:
df_vec = df_vec.drop(
    "genres_array",
    "production_countries_array",
    "production_companies_array",
    "spoken_languages_array",
    "cast_array",
    "director_array",
    "writers_array",
    "original_language_array",
    "runtime",
    "popularity",
    "release_year",
    "runtime_vec",
    "popularity_vec",
    "release_year_vec",
    "overview_tokens",
    "overview_filtered",
    "overview_tf",
    "overview"
)

In [250]:
# list 1000 release_year_scaled
# df_vec.select("release_year_scaled").show(10, truncate=True)

 
df_vec.show(5, truncate=True)

+-------+--------------------+--------------+---------------------------+---------------------------+-----------------------+--------------------+--------------------+--------------------+------------------------+--------------------+--------------------+-------------------+
|     id|     overview_vector| genres_vector|production_countries_vector|production_companies_vector|spoken_languages_vector|         cast_vector|     director_vector|      writers_vector|original_language_vector|      runtime_scaled|   popularity_scaled|release_year_scaled|
+-------+--------------------+--------------+---------------------------+---------------------------+-----------------------+--------------------+--------------------+--------------------+------------------------+--------------------+--------------------+-------------------+
|1301803|(10000,[78,1262,1...|(19,[1],[1.0])|           (250,[40],[1.0])|             (185922,[],[])|        (80,[32],[1.0])|      (262144,[],[])|      (262144,[],[])|     

In [251]:
final_assembler = VectorAssembler(
    inputCols=[
        "genres_vector",
        "production_countries_vector",
        "production_companies_vector",
        "spoken_languages_vector",
        "cast_vector",
        "director_vector",
        "writers_vector",
        "original_language_vector",
        "runtime_scaled",
        "popularity_scaled",
        "release_year_scaled",
        "overview_vector"
    ],
    outputCol="content_features"
)

df_vec_assembler = final_assembler.transform(df_vec)

In [252]:
df_vec_assembler.show(5, truncate=True)

+-------+--------------------+--------------+---------------------------+---------------------------+-----------------------+--------------------+--------------------+--------------------+------------------------+--------------------+--------------------+-------------------+--------------------+
|     id|     overview_vector| genres_vector|production_countries_vector|production_companies_vector|spoken_languages_vector|         cast_vector|     director_vector|      writers_vector|original_language_vector|      runtime_scaled|   popularity_scaled|release_year_scaled|    content_features|
+-------+--------------------+--------------+---------------------------+---------------------------+-----------------------+--------------------+--------------------+--------------------+------------------------+--------------------+--------------------+-------------------+--------------------+
|1301803|(10000,[78,1262,1...|(19,[1],[1.0])|           (250,[40],[1.0])|             (185922,[],[])|        

## 5. Chargement en base